In [ ]:
import os
import yaml
import kagglehub
from google.colab import drive

print("Mounting Google Drive...")
drive.mount('/content/drive')

print("Installing dependencies...")
os.system('pip install torch torchvision tqdm tensorboard kagglehub pandas')


Mounting Google Drive...
Mounted at /content/drive
Installing dependencies...


0

In [ ]:


!git clone https://github.com/Brayden-Zhang/GeoGuessr-AI.git
%cd GeoGuessr-AI


Enter your GitHub PAT: ··········
Cloning into 'GeoGuessr-AI'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 13 (delta 0), reused 13 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (13/13), done.
/content/GeoGuessr-AI


In [ ]:
import os

# Ensure the required directory exists
kaggle_config_dir = os.path.expanduser("~/.config/kaggle")
os.makedirs(kaggle_config_dir, exist_ok=True)

# Move kaggle.json to the correct location
if os.path.exists("kaggle.json"):  # Check if kaggle.json exists in the current directory
    os.system("mv kaggle.json ~/.config/kaggle/")  # Move the file
    os.system("chmod 600 ~/.config/kaggle/kaggle.json")  # Set permissions
    print("kaggle.json moved to ~/.config/kaggle/ and permissions set.")
else:
    print("kaggle.json not found in the current directory. Please upload it.")


kaggle.json moved to ~/.config/kaggle/ and permissions set.


In [ ]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile

# Proceed with dataset downloading
print("Downloading dataset...")
dataset_path = "/content/dataset"
os.makedirs(dataset_path, exist_ok=True)

# Specify the Kaggle dataset
kaggle_dataset = "paulchambaz/google-street-view"

# Ensure Kaggle API is installed
try:
    from kaggle.api.kaggle_api_extended import KaggleApi
except ImportError:
    os.system('pip install kaggle')
    from kaggle.api.kaggle_api_extended import KaggleApi

# Initialize Kaggle API
api = KaggleApi()
api.authenticate()

# Download the dataset
download_path = "/content/google-street-view"
api.dataset_download_files(kaggle_dataset, path=download_path, unzip=True)
print(f"Dataset downloaded to: {download_path}")



# Verify dataset contents
print("Dataset contents:")
print(os.listdir(dataset_path))


Dataset URL: https://www.kaggle.com/datasets/paulchambaz/google-street-view


KeyboardInterrupt: 

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, Subset, random_split
import torchvision.transforms as transforms
import pandas as pd
import os
from PIL import Image
from tqdm import tqdm

class GeoGuessrDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        # Read coordinates from CSV instead of .npy
        self.coords_df = pd.read_csv(os.path.join(data_dir, 'coords.csv'),
                                   header=None,
                                   names=['latitude', 'longitude'])

        # Default transform if none provided
        if transform is None:
            self.transform = transforms.Compose([
                transforms.Resize(256),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]
                )
            ])
        else:
            self.transform = transform

    def __len__(self):
        return len(self.coords_df)

    def __getitem__(self, idx):
        # Images are named as n.png
        img_path = os.path.join(self.data_dir, f'{idx}.png')

        # Load and transform image
        img = self.pil_loader(img_path)
        if self.transform:
            img = self.transform(img)

        # Get coordinates as target
        coords = self.coords_df.iloc[idx]
        target = torch.tensor([coords.latitude, coords.longitude], dtype=torch.float)

        return img, target

    @staticmethod
    def pil_loader(path: str) -> Image.Image:
        with open(path, "rb") as f:
            img = Image.open(f)
            return img.convert("RGB")


# Create full dataset
dataset_path = r"/content/google-street-view/dataset/"
full_dataset = GeoGuessrDataset(dataset_path)

# Compute split sizes for 80-10-10
dataset_size = len(full_dataset)
train_size = int(0.8 * dataset_size)
val_size = int(0.1 * dataset_size)
test_size = dataset_size - train_size - val_size

# Perform the split
train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



In [ ]:
import torch
import torch.nn as nn
class CNNModel(nn.Module):
   def __init__(self):
       super(CNNModel, self).__init__()

       # CNN Feature Extractor
       self.features = nn.Sequential(
           nn.Conv2d(3, 64, kernel_size=3, padding=1),
           nn.ReLU(inplace=True),
           nn.MaxPool2d(kernel_size=2, stride=2),

           nn.Conv2d(64, 128, kernel_size=3, padding=1),
           nn.ReLU(inplace=True),
           nn.MaxPool2d(kernel_size=2, stride=2),

           nn.Conv2d(128, 256, kernel_size=3, padding=1),
           nn.ReLU(inplace=True),
           nn.MaxPool2d(kernel_size=2, stride=2),
       )

       # Regression head
       self.regressor = nn.Sequential(
           nn.Flatten(),
           nn.Linear(256 * 32 * 32, 512),  # Adjust size based on your input image size
           nn.ReLU(),
           nn.Dropout(0.5),
           nn.Linear(512, 256),
           nn.ReLU(),
           nn.Dropout(0.5),
           nn.Linear(256, 2)  # 2 outputs for x, y coordinates
       )

   def forward(self, x):
       x = self.features(x)
       coordinates = self.regressor(x)
       return coordinates

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm


# Set up model, optimizer, and loss function
model = CNNModel()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# DataLoaders (replace these with your datasets)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Training configurations
num_epochs = 50
checkpoint_path = 'best_model.pth'
best_val_loss = float('inf')


# Training Loop
for epoch in range(1, num_epochs + 1):
    print(f"\nEpoch [{epoch}/{num_epochs}]")

    # Training Phase
    model.train()
    train_loss = 0.0
    train_bar = tqdm(train_loader, desc='Training', leave=False)
    for images, coordinates in train_bar:
        images, coordinates = images.to(device), coordinates.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, coordinates)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        train_bar.set_postfix({'Batch Loss': f"{loss.item():.4f}"})

    train_loss /= len(train_loader)
    print(f"Training Loss: {train_loss:.4f}")

    # Validation Phase
    model.eval()
    val_loss = 0.0
    val_bar = tqdm(val_loader, desc='Validation', leave=False)
    with torch.no_grad():
        for images, coordinates in val_bar:
            images, coordinates = images.to(device), coordinates.to(device)

            outputs = model(images)
            loss = criterion(outputs, coordinates)
            val_loss += loss.item()

            val_bar.set_postfix({'Batch Loss': f"{loss.item():.4f}"})

    val_loss /= len(val_loader)
    print(f"Validation Loss: {val_loss:.4f}")

    # Save the best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_loss,
        }, checkpoint_path)
        print("Saved new best validation loss model!")

# Testing Phase
print("\nStarting testing phase...")
model.eval()
test_loss = 0.0
test_bar = tqdm(test_loader, desc='Testing', leave=False)
with torch.no_grad():
    for images, coordinates in test_bar:
        images, coordinates = images.to(device), coordinates.to(device)

        outputs = model(images)
        loss = criterion(outputs, coordinates)
        test_loss += loss.item()

        test_bar.set_postfix({'Batch Loss': f"{loss.item():.4f}"})

test_loss /= len(test_loader)
print(f"\nTest Loss: {test_loss:.4f}")



Epoch [1/50]


Training Loss: 3257.7617


Validation Loss: 3124.0331
Saved new best validation loss model!

Epoch [2/50]


Training Loss: 3109.3034


Validation Loss: 2944.6327
Saved new best validation loss model!

Epoch [3/50]


...

Training Loss: 259.6995


Validation Loss: 2420.7204
Saved new best validation loss model!

Epoch [50/50]


Training Loss: 265.3537


Validation Loss: 2441.8248

Starting testing phase...



Test Loss: 2414.1823


In [ ]:
# torch.save(model.state_dict(), 'best_model.pth')
